In [27]:
import re
import pandas as pd
from pyBioInfo.IO.File import GtfFile

In [56]:
tid2gid = dict()
with GtfFile("results/assembly/taco/stringtie.gtf.gz") as f:
    for record in f:
        if record.feature == "transcript":
            tid = record.attributes["transcript_id"]
            gid = record.attributes["gene_id"]
            tid2gid[tid] = gid

In [57]:
with open("results/function/eggnog/out.emapper.annotations") as f:
    for line in f:
        if line.startswith("#query"):
            header = line.strip("\n").split("\t")
            break
            
dat = pd.read_csv("results/function/eggnog/out.emapper.annotations", 
                  sep="\t", comment="#", header=None, index_col=0)
dat.columns = header[1:]
dat.index.name = "query"
dat["isoform"] = [s.split(":")[0].split("_")[1] for s in dat.index]
dat["gene_id"] = [tid2gid[tid] for tid in dat["isoform"]]
dat.head()

,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,EC,...,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,isoform,gene_id
query,,,,,,,,,,,,,,,,,,,,,
lcl|ORF14_TU16:26:2797,215358.XP_010743573.1,0.000000e+00,963.0,"KOG3100@1|root,KOG3100@2759|Eukaryota,38EI3@33...",33208|Metazoa,S,"Deoxynucleotidyltransferase, terminal, interac...",DNTTIP2,"GO:0003674,GO:0003676,GO:0003723,GO:0005488,GO...",-,...,-,-,-,-,-,-,-,Fcf2,TU16,G11
lcl|ORF2_TU118:270:1232,303518.XP_005734101.1,4.290000e-231,642.0,"COG2866@1|root,KOG3641@2759|Eukaryota,38FGV@33...",33208|Metazoa,E,ATP GTP binding protein-like 4,AGBL4,"GO:0000003,GO:0002252,GO:0002376,GO:0003674,GO...",-,...,-,-,-,-,-,-,-,Peptidase_M14,TU118,G40
lcl|ORF14_TU118:4404:4595,215358.XP_010743578.1,1.150000e-28,112.0,"COG2866@1|root,KOG3641@2759|Eukaryota,38FGV@33...",33208|Metazoa,E,ATP GTP binding protein-like 4,AGBL4,"GO:0000003,GO:0002252,GO:0002376,GO:0003674,GO...",-,...,-,-,-,-,-,-,-,Peptidase_M14,TU118,G40
lcl|ORF60_TU118:5867:7198,144197.XP_008304851.1,3.300000e-209,591.0,"28JFY@1|root,2QRV3@2759|Eukaryota,38BJ0@33154|...",33208|Metazoa,S,Spermatogenesis-associated protein,SPATA6,"GO:0000003,GO:0003674,GO:0005488,GO:0005515,GO...",-,...,M00427,-,-,"ko00000,ko00001,ko00002,ko03019,ko03110,ko04121",-,-,-,SPATA6,TU118,G40
lcl|ORF2_TU117:405:1946,244447.XP_008334091.1,4.670000e-310,851.0,"COG2866@1|root,KOG3641@2759|Eukaryota,38FGV@33...",33208|Metazoa,E,ATP GTP binding protein-like 4,AGBL4,"GO:0000003,GO:0002252,GO:0002376,GO:0003674,GO...",-,...,-,-,-,-,-,-,-,Peptidase_M14,TU117,G40


In [97]:
temp = []
for gos in dat["GOs"]:
    gos = gos.split(",")
    for g in gos:
        if g == "-":
            continue
        temp.append(g)
temp = set(temp)
for g in temp:
    if g not in go_basic_list:
        print(g)

In [96]:
go_basic_list = []
with open("../common/go-basic.obo") as f:
    lines = None
    for line in f:
        line = line.strip("\n")
        if line == "[Term]":
            lines = [line]
        elif line == "":
            if lines:
                d = dict()
                for item in lines[1:]:
                    j = item.find(": ")
                    k, v = item[:j], item[j + 2:]
                    if k == "id":
                        go_basic_list.append(v)
                    elif k == "alt_id":
                        go_basic_list.append(v)
                lines = None
        else:
            if lines:
                lines.append(line)
go_basic_list = set(go_basic_list)

In [98]:
from bs4 import BeautifulSoup
import requests

In [99]:
obj = requests.get("https://www.kegg.jp/kegg/pathway.html")

In [104]:
bs = BeautifulSoup(obj.text)

In [113]:
n = 0
for item in bs.select("a"):
    href = item.get("href")
    text = item.text
    if re.match("/pathway/[a-z0-9]+", href):
        print(href, text, sep="\t")
        n += 1

/pathway/map01100	Metabolic pathways
/pathway/map01110	Biosynthesis of secondary metabolites
/pathway/map01120	Microbial metabolism in diverse environments
/pathway/map01200	Carbon metabolism
/pathway/map01210	2-Oxocarboxylic acid metabolism
/pathway/map01212	Fatty acid metabolism
/pathway/map01230	Biosynthesis of amino acids
/pathway/map01232	Nucleotide metabolism
/pathway/map01250	Biosynthesis of nucleotide sugars
/pathway/map01240	Biosynthesis of cofactors
/pathway/map01220	Degradation of aromatic compounds
/pathway/map00010	Glycolysis / Gluconeogenesis
/pathway/map00020	Citrate cycle (TCA cycle)
/pathway/map00030	Pentose phosphate pathway
/pathway/map00040	Pentose and glucuronate interconversions
/pathway/map00051	Fructose and mannose metabolism
/pathway/map00052	Galactose metabolism
/pathway/map00053	Ascorbate and aldarate metabolism
/pathway/map00500	Starch and sucrose metabolism
/pathway/map00520	Amino sugar and nucleotide sugar metabolism
/pathway/map00620	Pyruvate metabolism
/